In [1]:
# Imports
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import torchvision
import os
import pandas as pd
from tqdm import tqdm
from skimage import io
from skimage.transform import rescale, resize, downscale_local_mean
from torch.utils.data import (
    Dataset,
    DataLoader,
)  # Gives easier dataset managment and creates mini batches

/home/amax/.conda/envs/pyr/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip install torch-summary
!pip install tqdm

In [3]:
class ImagesDataProcess(Dataset):
    def __init__(self,csv_file,root_dir,tranform=None):
        self.annotation=pd.read_csv(csv_file)
        self.root_dir=root_dir
        self.tranform=tranform
        
    def __len__(self):
        return len(self.annotation)
    
    def __getitem__(self,index):
        img_path=os.path.join(self.root_dir,self.annotation.iloc[index,0])
        image=io.imread(img_path)
        image=resize(image,(250,250),anti_aliasing=True)
        y_label=torch.tensor(int(self.annotation.iloc[index,1]))
        
        if self.tranform:
            image=self.tranform(image)
        return (image,y_label)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
# Hyperparameters
in_channel = 3
num_classes = 3
learning_rate = 1e-3
batch_size = 32
num_epochs = 10
#root_dir_path='../input/cat-dog-images-for-classification/cat_dog'
root_dir_path="dataset"
#csv_file_path='../input/cat-dog-imagEes-for-classification/cat_dog.csv'
csv_file_path="us_GA.csv"

In [6]:
dataset=ImagesDataProcess(
    csv_file=csv_file_path,
    root_dir=root_dir_path,
    tranform=transforms.ToTensor()
)


In [7]:
len(dataset)

585

In [8]:
train_set, test_set = torch.utils.data.random_split(dataset, [468, 117])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)


In [9]:
# Model
model = torchvision.models.googlenet(pretrained=True)
model.fc=nn.Sequential(
    nn.Linear(in_features=1024,out_features=512),
    nn.ReLU(),
    nn.Linear(in_features=512,out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128,out_features=32),
    nn.ReLU(),
    nn.Linear(in_features=32,out_features=3,bias=True)
) 
model.to(device);
model

/home/amax/.conda/envs/pyr/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/home/amax/.conda/envs/pyr/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [10]:
from torchsummary import summary

print(summary(model, (3, 250, 250),verbose=0))

Layer (type:depth-idx)                   Output Shape              Param #
├─BasicConv2d: 1-1                       [-1, 64, 125, 125]        --
|    └─Conv2d: 2-1                       [-1, 64, 125, 125]        9,408
|    └─BatchNorm2d: 2-2                  [-1, 64, 125, 125]        128
├─MaxPool2d: 1-2                         [-1, 64, 62, 62]          --
├─BasicConv2d: 1-3                       [-1, 64, 62, 62]          --
|    └─Conv2d: 2-3                       [-1, 64, 62, 62]          4,096
|    └─BatchNorm2d: 2-4                  [-1, 64, 62, 62]          128
├─BasicConv2d: 1-4                       [-1, 192, 62, 62]         --
|    └─Conv2d: 2-5                       [-1, 192, 62, 62]         110,592
|    └─BatchNorm2d: 2-6                  [-1, 192, 62, 62]         384
├─MaxPool2d: 1-5                         [-1, 192, 31, 31]         --
├─Inception: 1-6                         [-1, 256, 31, 31]         --
|    └─BasicConv2d: 2-7                  [-1, 64, 31, 31]          --
|

In [11]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
# Train Network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in tqdm(enumerate(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device,dtype=torch.float)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")


15it [00:37,  2.50s/it]


Cost at epoch 0 is 0.8851838350296021


15it [00:36,  2.44s/it]


Cost at epoch 1 is 0.5746073424816132


15it [00:36,  2.45s/it]


...

Cost at epoch 7 is 0.2788371016581853


15it [00:36,  2.41s/it]


Cost at epoch 8 is 0.2535185694694519


15it [00:36,  2.41s/it]

Cost at epoch 9 is 0.1640549932916959


In [13]:
# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.to(device=device,dtype=torch.float)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()


In [14]:
print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)

Checking accuracy on Training Set


100%|██████████| 15/15 [00:35<00:00,  2.37s/it]


Got 316 / 468 with accuracy 67.52
Checking accuracy on Test Set


100%|██████████| 4/4 [00:08<00:00,  2.22s/it]

Got 73 / 117 with accuracy 62.39
